In [0]:
dbutils.widgets.text(name="env", defaultValue="dev")

In [0]:
catalog = "my_catalog"
env = schema = dbutils.widgets.get("env")

print(f"env    : {env}")
print(f"catalog: {catalog}")
print(f"schema : {schema}")

products_tbl       = f"{catalog}.{schema}.products"
orders_tbl         = f"{catalog}.{schema}.orders"
orders_refined_tbl = f"{catalog}.{schema}.orders_refined"
gold_tbl           = f"{catalog}.{schema}.gold_product_revenue"

In [0]:
import pyspark.sql.functions as F

orders_refined = spark.table(orders_refined_tbl)
products_revenue = (
    orders_refined
    .groupBy(["product_id", "product_name", "category"])
    .agg(
        F.sum("line_total").alias("total_revenue")
    )
    .orderBy(F.col("total_revenue").desc())
)

display(products_revenue)

In [0]:
(
    products_revenue.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_tbl)
)

print(f"Table {gold_tbl} is written.")